In [1]:
import os

SEASONS = list(range(2023,2025))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [ ]:
SEASONS

In [2]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [3]:
async def get_html(url, selector, sleep=10, retries=5):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [ ]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [10]:
url = f"https://www.basketball-reference.com/leagues/NBA_2024_games-november.html"
save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
html = await get_html(url, "#all_schedule")
with open(save_path, "w+") as f:
    f.write(html)

2023-24 NBA Schedule | Basketball-Reference.com


In [ ]:
for season in SEASONS:
    await scrape_season(season)

In [13]:
standings_files = os.listdir(STANDINGS_DIR)
standings_files

['NBA_2023_games-may.html',
 'NBA_2023_games-april.html',
 '.DS_Store',
 'NBA_2023_games-january.html',
 'NBA_2024_games-november.html',
 'NBA_2023_games-november.html',
 'NBA_2024_games-october.html',
 'NBA_2023_games-december.html',
 'NBA_2023_games-february.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-october.html']

In [14]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [15]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    print(files)
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

['NBA_2023_games-may.html', 'NBA_2023_games-april.html', 'NBA_2023_games-january.html', 'NBA_2023_games-november.html', 'NBA_2023_games-december.html', 'NBA_2023_games-february.html', 'NBA_2023_games-june.html', 'NBA_2023_games-march.html', 'NBA_2023_games-october.html']
['NBA_2024_games-november.html', 'NBA_2024_games-october.html']
Timeout error on https://www.basketball-reference.com/boxscores/202311090IND.html
Bucks vs Pacers, November 9, 2023 | Basketball-Reference.com
Hawks vs Magic, November 9, 2023 | Basketball-Reference.com
76ers vs Pistons, November 10, 2023 | Basketball-Reference.com
Hornets vs Wizards, November 10, 2023 | Basketball-Reference.com
Nets vs Celtics, November 10, 2023 | Basketball-Reference.com
Pelicans vs Rockets, November 10, 2023 | Basketball-Reference.com
Jazz vs Grizzlies, November 10, 2023 | Basketball-Reference.com
Timberwolves vs Spurs, November 10, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202311100